In [1]:
import os
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
from ultralytics import YOLO

In [ ]:
class YOLOImageProcessor(Node):
    def __init__(self):
        super().__init__('yolo_processor')
        self.bridge = CvBridge()

        # Load YOLO model (replace 'yolo11n_ncnn_model' with your specific weights file)
        self.model = YOLO('best_ncnn_model')

        # Subscription to the raw image topic
        self.subscription = self.create_subscription(
            Image,
            '/image_raw',  # Input topic
            self.image_callback,
            10
        )

        # Publisher for processed images
        self.publisher = self.create_publisher(
            Image,
            '/yolo_detected',  # Output topic
            10
        )

        # Timer to control processing frequency (1 Hz = 1 second)
        self.timer = self.create_timer(1.0, self.process_image)

        # Variable to store the latest image
        self.latest_image = None

        self.get_logger().info("YOLO Image Processor node initialized.")

    def image_callback(self, msg):
        # Store the latest image received
        try:
            self.latest_image = self.bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')
        except Exception as e:
            self.get_logger().error(f"Error converting ROS image to OpenCV format: {str(e)}")

    def process_image(self):
        # Process the latest image if available
        if self.latest_image is not None:
            try:
                # Flip the image vertically to correct the upside-down orientation
                flipped_image = cv2.flip(self.latest_image, -1)

                # Run YOLO detection
                processed_image = self.run_yolo(flipped_image)

                # Convert processed image back to ROS message
                ros_image = self.bridge.cv2_to_imgmsg(processed_image, encoding='bgr8')

                # Publish the processed image
                self.publisher.publish(ros_image)

                self.get_logger().info("Processed and published image.")
            except Exception as e:
                self.get_logger().error(f"Error processing image: {str(e)}")

    def run_yolo(self, frame):
        # Perform YOLO inference
        results = self.model(frame)  # Run YOLO on the input frame

        # Draw bounding boxes and labels
        annotated_frame = frame.copy()
        for result in results[0].boxes:  # Access detected objects
            x1, y1, x2, y2 = map(int, result.xyxy[0])  # Bounding box coordinates
            confidence = float(result.conf[0])  # Confidence score
            class_id = int(result.cls[0])  # Class ID
            label = f"{self.model.names[class_id]} {confidence:.2f}"

            # Draw the bounding box and label
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(
                annotated_frame,
                label,
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                2
            )

        return annotated_frame


def main(args=None):
    rclpy.init(args=args)
    node = YOLOImageProcessor()
    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        node.get_logger().info("Shutting down.")
    finally:
        node.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()
